In [ ]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
# image_source: image source for inference
#     path to image file
#     URL of image
#     PIL image object
#     numpy array

hw_location = "@cloud"
model_zoo_url = "https://cs.degirum.com/degirum/timm_gender_model_test"
text_det_model_name = "text_detection--704x704_float_openvino_cpu_1"
text_rec_model_name = "resnet_text_recognition--704x704_float_openvino_cpu_1"
image_source = "https://raw.githubusercontent.com/DeGirum/PySDKExamples/features/images/demo_text_det.jpg"


In [ ]:
import degirum as dg, degirum_tools
from postprocessor_ocr_det import OCRTextDetPostprocessor
from preprocessor_ocr_rec import OCRTextRecPreprocessor
from postprocessor_ocr_rec import OCRTextRecPostprocessor
# Connect to AI inference engine
# degirum_cloud_token: Degirum cloud API access token
text_det_zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())
text_rec_zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())

In [ ]:
# load text detection AI model and text recognition AI model
text_det_model = text_det_zoo.load_model(text_det_model_name, 
                                         image_backend = "opencv", 
                                         overlay_show_labels = False, 
                                         custom_postprocessor = OCRTextDetPostprocessor
                                         )
text_rec_model = text_rec_zoo.load_model(text_rec_model_name, 
                                         custom_postprocessor = OCRTextRecPostprocessor
                                         )

rec_H, rec_W = text_rec_model.model_info.InputW[0],text_rec_model.model_info.InputC[0]
preprocessor = OCRTextRecPreprocessor(rec_H, rec_W)

text_det_res = text_det_model(image_source)
text_rec_preprocess = preprocessor.initialize(text_det_res)
for image_crop, index in text_rec_preprocess:
    result = text_rec_model([image_crop])
    text_det_res.results[index]["label"] = result.results[0]["label"]

In [ ]:
text_det_res.image_overlay